# Apache Hudi Core Conceptions (2) - COW: File Layouts & File Sizing

*Author: [Laurence Geng](https://laurence.blog.csdn.net) @ [https://laurence.blog.csdn.net](https://laurence.blog.csdn.net)*

## 1. Configuration

In [1]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-16 01:17 /tmp/hudi-spark-bundle.jar


In [2]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
80,application_1678096020253_0135,spark,idle,Link,Link,None,


In [3]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [4]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
81,application_1678096020253_0140,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions

env: WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions


In [6]:
%%sh
# make workspace
mkdir -p $WORKSPACE
# deploy hudi-stat.sh, a utility shell script to output hudi table status
wget https://raw.githubusercontent.com/bluishglc/apache-hudi-core-conceptions/master/hudi-stat.sh -O $WORKSPACE/hudi-stat.sh &>/dev/null
chmod a+x $WORKSPACE/hudi-stat.sh
ls $WORKSPACE/hudi-stat.sh

/home/emr-notebook/apache-hudi-core-conceptions/hudi-stat.sh


In [7]:
%%html
<style>
table {float:left}
</style>

## 2. Test Case 1 - COW File Layouts

### 2.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+5 Small Files
2|Insert|14.6MB|+1 Big File
3|Insert|3.7MB|+1 Small File
4|Insert|188.5MB|+1 Big File, +1 Small File

### 2.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.parquet.max.file.size|125829120 ( 120MB )|Default Value
hoodie.parquet.small.file.limit|104857600 ( 100MB )|Default Value
hoodie.copyonwrite.record.size.estimate|1024|175

### 2.3. Set Variables

In [8]:
%%sql
set TABLE_NAME=reviews_cow_layouts_sizing_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%env TABLE_NAME=reviews_cow_layouts_sizing_1

env: TABLE_NAME=reviews_cow_layouts_sizing_1


### 2.4. Create Table

In [10]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [11]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.5. Insert 96MB / +1 Small File

In [13]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316011817174 │ commit │ COMPLETED │ 03-16 01:18 │ 03-16 01:18 │ 03-16 01:19 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

### 2.6. Insert 14.6MB / +1 Big File

In [15]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1998;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316011817174 │ commit │ COMPLETED │ 03-16 01:18 │ 03-16 01:18 │ 03-16 01:19 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316011934548 │ commit │ COMPLETED │ 03-16 01:19 │ 03-16 01:19 │ 03-16 01:20 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde

### 2.7. Insert 3.7MB / +1 Small File

In [17]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1997;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [18]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316011817174 │ commit │ COMPLETED │ 03-16 01:18 │ 03-16 01:18 │ 03-16 01:19 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316011934548 │ commit │ COMPLETED │ 03-16 01:19 │ 03-16 01:19 │ 03-16 01:20 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230316012039888 │ commit │ COMPLETED │ 03-16 01:20 │ 03-16 01:20 │ 03-16 01:20 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤══════════════════

### 2.8. Insert 188.5MB / +1 Max File +1 Small File

In [19]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316011817174 │ commit │ COMPLETED │ 03-16 01:18 │ 03-16 01:18 │ 03-16 01:19 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316011934548 │ commit │ COMPLETED │ 03-16 01:19 │ 03-16 01:19 │ 03-16 01:20 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230316012039888 │ commit │ COMPLETED │ 03-16 01:20 │ 03-16 01:20 │ 03-16 01:20 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230316012115748 │ commit │ COMPLETED │ 03-16 01:21 │ 03-16 01:21 

## 3. Test Case 2 - COW Default Settings

### 3.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|96MB|+1 Small File
2|Insert|417MB|+3 Max Files + 1 Small File

### 3.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.parquet.max.file.size|125829120 ( 120MB )|Default Value
hoodie.parquet.small.file.limit|104857600 ( 100MB )|Default Value
hoodie.copyonwrite.record.size.estimate|1024|Default Value

### 3.3. Set Variables

In [21]:
%%sql
set TABLE_NAME=reviews_cow_layouts_sizing_2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [22]:
%env TABLE_NAME=reviews_cow_layouts_sizing_2

env: TABLE_NAME=reviews_cow_layouts_sizing_2


### 3.4. Create Table

In [23]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [24]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [25]:
%%sql
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating int, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp'
    -- hoodie.copyonwrite.record.size.estimate = '175'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 3.5. Insert 96MB / +5 Small Files

In [26]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2003;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316012249944 │ commit │ COMPLETED │ 03-16 01:22 │ 03-16 01:23 │ 03-16 01:23 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

### 3.6. Insert 417MB / +3 Max Files + 1 Small File

In [28]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2010;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%%sh
${WORKSPACE}/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230316012249944 │ commit │ COMPLETED │ 03-16 01:22 │ 03-16 01:23 │ 03-16 01:23 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230316012337424 │ commit │ COMPLETED │ 03-16 01:23 │ 03-16 01:24 │ 03-16 01:25 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde